# Eksperimantiranje

Prvo ćemo importati potrebne pakete i fiksirati _random seed_ kako bi kod bio reproducibilan.

In [27]:
from enum import Enum
from time import sleep

import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

random_state = 42
rng = np.random.RandomState(random_state)

Sljedeća funkcija služi generiranju uzorka jediničnih normalnih veličina $x$ i $y$ td.
\begin{align*}
y &= \beta x + \varepsilon, \ \ \varepsilon \sim N(0, \sigma^2) \\
\mathrm{Cov}(x_i, x_j) &= \sigma^2 > 0, \ \ i \neq j.
\end{align*}

In [3]:
def get_sample(beta: np.array, rho: float, sample_size: int, error_var: float = 1):

    """Generates sample of X and y such that y = X*beta + eps, eps ~ N(0, sqrt(error_var))

    Args:
        beta (np.array): Linear transformation vector.
        rho (float): Covariance between covariates.

    Returns:
        sample: generated sample.
    """

    N = beta.shape[0]
    cov = rho * np.ones([N, N])
    cov = cov + np.diag(1-rho * np.ones(N))

    N = beta.shape[0]
    X = rng.multivariate_normal(mean = np.zeros(N), cov=cov, size=sample_size)

    error = rng.normal(loc=0, scale=np.sqrt(error_var), size=X.shape[0])
    y = np.matmul(X, beta) + error
    
    return X, y

Brzinski _sanity check_: ako je $\rho = 0,$ tada je $$\mathrm{Var}(y) = \sum_{k=0}^N \beta_k^2 \mathrm{Var}(X_k) + \mathrm{Var}(\varepsilon),$$ što bi u slucaju $N = 3, \beta = (1, \dots, 1)$ moralo biti $4$. Zaista, za dovoljno velik uzorak imamo da je uzoračka varijanca $\approx 4$:

In [4]:
N = 3
beta = np.ones(3)
rho = 0
sample_size = 1000000

X, y = get_sample(beta=beta, rho=rho, sample_size=sample_size)
print(f'>>> Var(y) = {y.var(ddof=1)}')

>>> Var(y) = 3.9966141119858167


Nastavljamo definiranjem funkcija za treniranje LS, PCR i PLS modela, pri čemu potonja dva kao parametar primaju i broj glavnih komponenti koje koriste.

In [5]:
class Model(str, Enum):
    linreg = "linreg"
    pcr = "pcr"
    pls = "pls"

    @staticmethod
    def train(model_name: str, X: np.array, y: np.array, n_components: int | None = None):

        if model_name not in list(Model):
            raise ValueError(f'No such model. Available models are {list(Model)}')
        
        if model_name == Model.linreg:
            model = LinearRegression()
        
        elif model_name == Model.pcr:
            model = make_pipeline(PCA(n_components=n_components, random_state=random_state), LinearRegression())
        
        elif model_name == Model.pls:
            model = PLSRegression(n_components=n_components)
        
        model.fit(X, y)

        return model

Jedan razuman način validacije naših modela bio bi da koristeći distribucije iz kojih smo generirali podatke izračunamo populacijski $\beta$ pa za grešku modela uzmemo
koliko se njegov koeficijent razlikuje od populacijskog, tj. ako je nas model dan s $y = \hat{\beta}x$, njegovu grešku možemo računati kao
\begin{align*}
\mathrm{Err}(\mathrm{Model}) = \|\beta-\hat{\beta}\|.
\end{align*}
Međutim, kako je prilikom visoke korelacije kovarijata taj $\beta$ "nestabilan", mi ćemo umjesto toga testirati naše modele na velikom testnom uzorku. Preciznije, prvo ćemo izgenerirati jako velik uzorak, zatim trenirati model na njegovom malom dijelu, a na ostatku izračunati $R2,$ što ce nam biti primarna metrika za validaciju modela. Takav pristup ima dvije prednosti:
1. Metrike poput kvaratne greške i $R2$ su interpretabilnije od udaljenosti do stvarnog $\beta$.
2. Tako se stvari rade u praksi (jer ne znamo stvarne distribucije pa ni vrijednost populacijskog koeficijenta); istrenira se model na uzorku koji nam je dan, a zatim validira na testnom skupu pa ide u produkciju.



In [6]:
SAMPLE_SIZE = 50_000

In [28]:
def train_and_evaluate_all_models(X_train: np.array, X_test: np.array, y_train: np.array, y_test: np.array, n_components: int):
    
    """Trains LS, PLS and PCR models on X_train and y_test and evaluates them on X_test and y_test.

    Args:
        n_components (int): Number of relevant components to use for predicition in PCR and PLS.

    Returns:
        dict: Contains model names with their respective R2 values.
    """

    score_dict = {'R2': {}, 'MSE': {}}

    for model_name in list(Model):
        model = Model.train(model_name=model_name, X=X_train, y=y_train, n_components=n_components)
        score_dict['R2'][model_name.value] = model.score(X_test, y_test)
        preds_test = model.predict(X_test)
        score_dict['MSE'][model_name.value] = mean_squared_error(preds_test, y_test)

    return score_dict

Donji primjer pokazuje kako kod "skoro" nezavisnih kovarijata PLS bolje predviđa nego PCR. To je i očekivano, pogotovo ako je broj komponenti puno manji od $N$ jer tada PCA nužno gubi bitne informacije za predviđanje. S druge strane, PLS, rastavljajuci zavisnu varijablu skupa s nezavisnima, ne izgubi gotovo ništa te predviđa jednako dobro kao i linearna regresija, ali u puno manjoj dimenziji pa je stoga interpretabilniji od nje.

In [29]:
N = 300
train_sample_size = 1000
n_components = 5
rho= 0.1

beta = rng.normal(loc=0, scale=25, size=N)
X, y = get_sample(beta=beta, rho=rho, sample_size=SAMPLE_SIZE)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_state, train_size=train_sample_size)

In [30]:
score_dict = train_and_evaluate_all_models(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, n_components=n_components)
score_dict

{'R2': {'linreg': 0.9999910704911152,
  'pcr': 0.0446818963982758,
  'pls': 0.9943686745783412},
 'MSE': {'linreg': 1.454770247950533,
  'pcr': 155637.71450221376,
  'pls': 917.4395575053132}}

In [31]:
df = pd.DataFrame(score_dict)
df.index.name = 'model'
df = df.reset_index()
df

,model,R2,MSE
0,linreg,0.999991,1.454770
1,pcr,0.044682,155637.714502
2,pls,0.994369,917.439558


Sad ćemo za razne parametre provesti eksperiment i podatke o R2 spremiti u datoteku _scores.csv_.

In [43]:
SAMPLE_SIZE = 100_000
index_columns = ['train_sample_size', 'N', 'n_components', 'rho']
columns = ['model', 'R2', 'MSE']
df = pd.DataFrame(None, columns=index_columns+columns)

Ns = [500, 100, 50, 10, 5]
loader = tqdm(Ns)

N_train_sample_size_ratios = [10, 5, 3, 2, 1]
n_components_N_ratios = [0.01, 0.1, 0.25, 0.5]
rhos = [0.01, 0.1, 0.2, 0.5, 0.9, 0.99]

for N in loader:
    for rho in rhos:
        
        beta = rng.normal(loc=0, scale=25, size=N)
        X, y = get_sample(beta=beta, rho=rho, sample_size=SAMPLE_SIZE)

        for train_sample_size in [N*x for x in N_train_sample_size_ratios]:

            X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_state, train_size=train_sample_size)

            for n_components in [int(N*x) for x in n_components_N_ratios]:

                if n_components == 0:
                    continue

                score_dict = train_and_evaluate_all_models(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, n_components=n_components)
                score_df = pd.DataFrame(score_dict)
                score_df.index.name = 'model'
                score_df = score_df.reset_index()
                for col_name, value in zip(index_columns, [train_sample_size, N, n_components, rho]):
                    score_df[col_name] = value
                print(df)
                df = pd.concat([df, score_df])
                loader.set_postfix(**score_dict['R2'] | {'rho': rho, 'N': N})
        
        score_df.to_csv('scores2.csv', index=False)

df.to_csv('scores2.csv', index=False)

  0%|          | 0/5 [00:05<?, ?it/s, N=500, linreg=1, pcr=0.00857, pls=1, rho=0.01]

Empty DataFrame
Columns: [train_sample_size, N, n_components, rho, model, R2, MSE]
Index: []


  0%|          | 0/5 [00:12<?, ?it/s, N=500, linreg=1, pcr=0.127, pls=1, rho=0.01]  

  train_sample_size    N n_components   rho   model        R2            MSE
0              5000  500            5  0.01  linreg  0.999996       1.120479
1              5000  500            5  0.01     pcr  0.008567  278123.542607
2              5000  500            5  0.01     pls  0.999916      23.445148


  0%|          | 0/5 [00:21<?, ?it/s, N=500, linreg=1, pcr=0.278, pls=1, rho=0.01]

...

   train_sample_size    N n_components   rho   model        R2            MSE
0               5000  500            5  0.01  linreg  0.999996       1.120479
1               5000  500            5  0.01     pcr  0.008567  278123.542607
2               5000  500            5  0.01     pls  0.999916      23.445148
0               5000  500           50  0.01  linreg  0.999996       1.120479
1               5000  500           50  0.01     pcr  0.127465  244769.316087
..               ...  ...          ...   ...     ...       ...            ...
1                  5    5            2  0.50     pcr  0.968577     155.697135
2                  5    5            2  0.50     pls  0.982283      87.783188
0                 50    5            1  0.90  linreg  0.997268       1.344059
1                 50    5            1  0.90     pcr  0.224494     381.522608
2                 50    5            1  0.90     pls  0.256323     365.864149

[1383 rows x 7 columns]
   train_sample_size    N n_components 

 80%|████████  | 4/5 [16:24<02:32, 152.18s/it, N=5, linreg=0.994, pcr=0.889, pls=0.989, rho=0.99]

   train_sample_size    N n_components   rho   model        R2            MSE
0               5000  500            5  0.01  linreg  0.999996       1.120479
1               5000  500            5  0.01     pcr  0.008567  278123.542607
2               5000  500            5  0.01     pls  0.999916      23.445148
0               5000  500           50  0.01  linreg  0.999996       1.120479
1               5000  500           50  0.01     pcr  0.127465  244769.316087
..               ...  ...          ...   ...     ...       ...            ...
1                 10    5            1  0.90     pcr  0.216806     385.253353
2                 10    5            1  0.90     pls  0.223029     382.192395
0                 10    5            2  0.90  linreg  0.996926       1.512256
1                 10    5            2  0.90     pcr  0.218512     384.414219
2                 10    5            2  0.90     pls  0.914604      42.006123

[1404 rows x 7 columns]
   train_sample_size    N n_components 

100%|██████████| 5/5 [16:24<00:00, 196.89s/it, N=5, linreg=0.994, pcr=0.965, pls=0.971, rho=0.99]


   train_sample_size    N n_components   rho   model        R2            MSE
0               5000  500            5  0.01  linreg  0.999996       1.120479
1               5000  500            5  0.01     pcr  0.008567  278123.542607
2               5000  500            5  0.01     pls  0.999916      23.445148
0               5000  500           50  0.01  linreg  0.999996       1.120479
1               5000  500           50  0.01     pcr  0.127465  244769.316087
..               ...  ...          ...   ...     ...       ...            ...
1                 25    5            2  0.99     pcr  0.889062      32.516059
2                 25    5            2  0.99     pls  0.989174       3.173164
0                 15    5            1  0.99  linreg  0.993213       1.989213
1                 15    5            1  0.99     pcr  0.887848      32.873156
2                 15    5            1  0.99     pls  0.887754      32.900725

[1425 rows x 7 columns]
   train_sample_size    N n_components 